In [1]:
import numpy as np
import pandas as pd

from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_predict, KFold

In [2]:
data = pd.read_csv('K:/Google Drive/DOUTORADO/Tese 2.0/Chapter I/KELLOGs/dataset.csv')

Elements = ['As', 'Ba', 'Cd', 'Co', 'Cr', 'Cu', 'Pb', 'Zn', 'Mo']

filter = pd.read_csv('filter.csv')

print(data.shape)
data = data[data['id.layer_uuid_c'].isin(filter['id.layer_uuid_c'])]
print(data.shape)

X = data.iloc[:,10:]

(1337, 1711)
(1165, 1711)


In [3]:
def calculate_metrics(y_true, y_pred):

    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    deviation = np.std(y_true - y_pred)
    rpd = np.std(y_true) / deviation if deviation != 0 else np.inf

    # Calculate interquartile distance
    iqr = np.percentile(y_true, 75) - np.percentile(y_true, 25)

    # Calculate RPIQ
    rpiq = np.std(y_true) / iqr if iqr != 0 else np.inf

    return mse, rmse, r2, rpd, rpiq

In [4]:
def pls_cross_validation(X, y, n_components_list):

    metrics_dict = {'mse': [], 'rmse': [], 'r2': [], 'rpd': [], 'rpiq': []}

    for n_components in n_components_list:
        pls_model = PLSRegression(n_components=n_components)
        y_pred = cross_val_predict(pls_model, X, y, cv=KFold(n_splits=10, shuffle=True, random_state=42))

        mse, rmse, r2, rpd, rpiq = calculate_metrics(y, y_pred)

        metrics_dict['mse'].append(mse)
        metrics_dict['rmse'].append(rmse)
        metrics_dict['r2'].append(r2)
        metrics_dict['rpd'].append(rpd)
        metrics_dict['rpiq'].append(rpiq)
        
    metrics_df = pd.DataFrame(metrics_dict)
    metrics_df['n_components'] = n_components_list
    metrics_df = metrics_df[['n_components', 'mse', 'rmse', 'r2', 'rpd', 'rpiq']]

    return metrics_df

In [5]:
def pls_validation(X_train, y_train, X_test, y_test, n_components, element):
    
    pls_model = PLSRegression(n_components=n_components)
    pls_model.fit(X_train, y_train)

    y_pred = pls_model.predict(X_test)

    metrics_dict = {'element': [], 'mse': [], 'rmse': [], 'r2': [], 'rpd': [], 'rpiq': []}
    
    mse, rmse, r2, rpd, rpiq = calculate_metrics(y_test, y_pred)

    metrics_dict['element'].append(element)
    metrics_dict['mse'].append(mse)
    metrics_dict['rmse'].append(rmse)
    metrics_dict['r2'].append(r2)
    metrics_dict['rpd'].append(rpd)
    metrics_dict['rpiq'].append(rpiq)
        
    metrics_df = pd.DataFrame(metrics_dict)
    metrics_df = metrics_df[['element', 'mse', 'rmse', 'r2', 'rpd', 'rpiq']]

    return metrics_df

In [6]:
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

In [7]:
dataframes = []

for i in range(1,10):
    y = data.iloc[:,i].values
    # Specify the list of numbers of components to try
    n_components_list = [2, X.shape[1] // 4, X.shape[1] // 2, X.shape[1]]
    df = pls_cross_validation(X, y, n_components_list)
    df['Element'] = data.iloc[:,i].name

    dataframes.append(df)

In [8]:
results = pd.concat(dataframes, ignore_index=True)

results.to_csv('pls_cv.csv')

In [9]:
unique_elements = results['Element'].unique()

best_ncomps = pd.DataFrame()

for element in unique_elements:
    element_results = results[results['Element'] == element]
    best_ncomp = element_results.loc[element_results['rmse'].idxmin()]
    best_ncomps = best_ncomps.append(best_ncomp, ignore_index=True)

best_ncomps

C:\Users\Angelo\AppData\Local\Temp\ipykernel_18824\3626035875.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  best_ncomps = best_ncomps.append(best_ncomp, ignore_index=True)


,n_components,mse,rmse,r2,rpd,rpiq,Element
0,2,1.240378,1.113723,0.343772,0.858397,1.685789,As
1,2,3856.899282,62.103939,0.219366,0.900928,0.734057,Ba
2,2,0.019786,0.140661,0.362745,0.853580,0.678048,Cd
3,425,0.642659,0.801660,0.407374,0.664159,1.184266,Co
4,850,7.754713,2.784728,0.469594,0.763184,35.781404,Cr
5,850,4.651754,2.156792,0.308096,0.658359,1.067868,Cu
6,850,1.340782,1.157921,0.377455,0.662036,0.714961,Pb
7,2,3.573071,1.890257,0.150803,0.928815,1.393897,Zn
8,2,0.024927,0.157882,0.217301,0.902969,1.086692,Mo


In [10]:
validation_df = []
count = 0
for element in unique_elements:
    n_components = int(best_ncomps[best_ncomps['Element'] == element]['n_components'].values)
    count += 1
    y = data.iloc[:,count].values
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    validation_metrics = pls_validation(X_train, y_train, X_test, y_test, n_components, element)

    validation_df.append(validation_metrics)

In [11]:
results_val = pd.concat(validation_df, ignore_index=True)

results_val.to_csv('pls_val.csv')

In [12]:
results_val

,element,mse,rmse,r2,rpd,rpiq
0,As,0.937203,0.968092,0.363621,0.836836,1.854965
1,Ba,4253.975584,65.222508,0.188112,0.897761,0.693979
2,Cd,0.019200,0.138563,0.335625,0.850721,0.683246
3,Co,0.654809,0.809203,0.496446,0.642911,1.374317
4,Cr,14.673420,3.830590,0.607470,0.806803,59.349808
5,Cu,3.523214,1.877023,0.470951,0.679317,1.001057
6,Pb,1.001395,1.000697,0.459157,0.659767,0.622284
7,Zn,2.410375,1.552538,0.181407,0.907610,1.175053
8,Mo,0.026089,0.161520,0.209237,0.904068,1.102982
